In [1]:
from neo4j import GraphDatabase
import osmnx as ox
import folium
import warnings
warnings.filterwarnings("ignore")


# Connexion à la base de données Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "12345678"))
start_point = (34.047608901290324, -4.959777988230495)

# Test de connexion
with driver.session() as session:
    result = session.run("RETURN 1")
    print("Connexion réussie" if result.single()[0] == 1 else "Connexion échouée")


Connexion réussie


In [ ]:
from geopy.geocoders import Nominatim

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        print("Impossible de trouver les coordonnées pour cet emplacement.")
        return None, None


# Utilisation de la fonction pour obtenir les coordonnées
latitude, longitude = get_coordinates("Fes")
if latitude is not None and longitude is not None:
    print("Latitude:", latitude)
    print("Longitude:", longitude)

In [4]:
def import_data_to_neo4j(driver, start_point, distance=1000):
    # Récupération des pharmacies
    pharmacies = ox.geometries_from_point(start_point, tags={'amenity': 'pharmacy'}, dist=distance)
    # Récupération du réseau routier
    G = ox.graph_from_point(start_point, dist=distance, network_type='drive')

    def create_pharmacy_node(tx, id, name, lat, lon):
        query = (
            "CREATE (p:Pharmacy {id: $id, name: $name, latitude: $lat, longitude: $lon})"
        )
        tx.run(query, id=id[1], name=name, lat=lat, lon=lon)

    def create_road_node(tx, osmid, lat, lon):
        query = (
            "CREATE (r:Road {osmid: $osmid, latitude: $lat, longitude: $lon})"
        )
        tx.run(query, osmid=osmid, lat=lat, lon=lon)

    def create_road_relationship(tx, from_osmid, to_osmid, length):
        query = (
            "MATCH (r1:Road {osmid: $from_osmid}), (r2:Road {osmid: $to_osmid}) "
            "CREATE (r1)-[:CONNECTED {length: $length}]->(r2)"
        )
        tx.run(query, from_osmid=from_osmid, to_osmid=to_osmid, length=length)

    with driver.session() as session:
        # Clear existing data
        session.run("MATCH (n) DETACH DELETE n")

        # Import pharmacies
        for idx, row in pharmacies.iterrows():
            session.execute_write(create_pharmacy_node, idx, row['name'], row.geometry.y, row.geometry.x)

        # Import road network nodes
        for node, data in G.nodes(data=True):
            session.execute_write(create_road_node, node, data['y'], data['x'])

        # Import road network edges
        for u, v, data in G.edges(data=True):
            session.execute_write(create_road_relationship, u, v, data['length'])

# Test de la fonction d'importation
import_data_to_neo4j(driver, start_point)
print("Données importées avec succès dans Neo4j")

Données importées avec succès dans Neo4j


In [5]:
def find_nearest_pharmacy(driver, start_point):
    query = (
        "MATCH (p:Pharmacy) "
        "RETURN p.id AS id, p.name AS name, p.latitude AS lat, p.longitude AS lon, "
        "point.distance(point({latitude: $lat, longitude: $lon}), point({latitude: p.latitude, longitude: p.longitude})) AS distance "
        "ORDER BY distance ASC LIMIT 1"
    )

    with driver.session() as session:
        result = session.run(query, lat=start_point[0], lon=start_point[1])
        return result.single()

# Test de la fonction de recherche de la pharmacie la plus proche
nearest_pharmacy = find_nearest_pharmacy(driver, start_point)
print(f"La pharmacie la plus proche est : {nearest_pharmacy['name']} à {nearest_pharmacy['distance']} mètres")


La pharmacie la plus proche est : Pharmacie Al Hanae صيدلية الهناء à 156.41452209294732 mètres


In [6]:
def create_map(start_point, pharmacy):
    m = folium.Map(location=start_point, zoom_start=14)

    # Add start point marker
    folium.Marker(
        location=start_point,
        popup="Start Point",
        icon=folium.Icon(color='green', icon='home'),
    ).add_to(m)

    # Add pharmacy marker
    folium.Marker(
        location=(pharmacy['lat'], pharmacy['lon']),
        popup=pharmacy['name'],
        icon=folium.Icon(color='red', icon='flag')
    ).add_to(m)

    return m

# Test de la fonction de création de carte
if nearest_pharmacy:
    m = create_map(start_point, nearest_pharmacy)
m

In [ ]:
def find_shortest_path(driver, start_point, nearest_pharmacy):
    start_node_query = (
        "MATCH (r:Road) "
        "RETURN r.osmid AS osmid, "
        "point.distance(point({latitude: $lat, longitude: $lon}), point({latitude: r.latitude, longitude: r.longitude})) AS distance "
        "ORDER BY distance ASC LIMIT 1"
    )
    
    with driver.session() as session:
        start_node_result = session.run(start_node_query, lat=start_point[0], lon=start_point[1])
        start_node = start_node_result.single()["osmid"]
        
        end_node_query = (
            "MATCH (r:Road) "
            "RETURN r.osmid AS osmid, "
            "point.distance(point({latitude: $lat, longitude: $lon}), point({latitude: r.latitude, longitude: r.longitude})) AS distance "
            "ORDER BY distance ASC LIMIT 1"
        )
        
        end_node_result = session.run(end_node_query, lat=nearest_pharmacy['lat'], lon=nearest_pharmacy['lon'])
        end_node = end_node_result.single()["osmid"]

        path_query = (
            "MATCH (start:Road {osmid: $start_osmid}), (end:Road {osmid: $end_osmid}), "
            "p = shortestPath((start)-[:CONNECTED*]-(end)) "
            "RETURN p"
        )
        
        path_result = session.run(path_query, start_osmid=start_node, end_osmid=end_node)
        path = path_result.single()["p"]
        return path

# Test de la fonction de recherche du chemin le plus court
if nearest_pharmacy:
    shortest_path = find_shortest_path(driver, start_point, nearest_pharmacy)
    print("Chemin le plus court trouvé")

def create_map(start_point, pharmacy, path, driver):
    m = folium.Map(location=start_point, zoom_start=14)

    # Add start point marker
    folium.Marker(
        location=start_point,
        popup="Start Point",
        icon=folium.Icon(color='green', icon='home'),
    ).add_to(m)

    # Add pharmacy marker
    folium.Marker(
        location=(pharmacy['lat'], pharmacy['lon']),
        popup=pharmacy['name'],
        icon=folium.Icon(color='red', icon='flag')
    ).add_to(m)

    # Add path to map
    latlons = []
    for segment in path.relationships:
        query = (
            "MATCH (r1:Road)-[rel:CONNECTED]->(r2:Road) "
            "WHERE ID(rel) = $rel_id "
            "RETURN r1.latitude AS lat1, r1.longitude AS lon1, r2.latitude AS lat2, r2.longitude AS lon2"
        )
        result = driver.session().run(query, rel_id=segment.id)
        segment_data = result.single()
        latlons.append((segment_data["lat1"], segment_data["lon1"]))
        latlons.append((segment_data["lat2"], segment_data["lon2"]))

    folium.PolyLine(latlons, color="blue", weight=2.5, opacity=1).add_to(m)
    
    return m

# Test de la fonction de création de carte
if nearest_pharmacy and shortest_path:
    m = create_map(start_point, nearest_pharmacy, shortest_path, driver)
m